请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

In [85]:
# ## baseline3版本，不参与建模的特征 ['os']
# ##添加特征dev_area，version_osv 但分数没有变化,去掉dev_height和dev_width后，涨了一点
# ## Score = 88.3027
#加入祖传参数，n_estimators=3000棵树后 score=88.4473
#lgb五折交叉验证， socre=88.2367
#xgb五折交叉验证， socre=
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [86]:
# 数据加载
train = pd.read_csv('/home/aistudio/data/data97586/train.csv')
test = pd.read_csv('/home/aistudio/data/data97586/test1.csv')


In [87]:
# 字符串类型 需要转换为数值（labelencoder）
object_cols = train.select_dtypes(include='object').columns
# 缺失值个数
temp = train.isnull().sum()
# 有缺失值的字段： lan, osv
temp[temp>0]


lan    183280
osv      6561
dtype: int64

In [88]:
# ['os', 'osv', 'lan', 'sid’]
features = train.columns.tolist()
features.remove('label')
print(features)


['Unnamed: 0', 'android_id', 'apptype', 'carrier', 'dev_height', 'dev_ppi', 'dev_width', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package', 'sid', 'timestamp', 'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type']


In [89]:
for feature in features:
    print(feature, train[feature].nunique())

Unnamed: 0 500000
android_id 362258
apptype 89
carrier 5
dev_height 798
dev_ppi 92
dev_width 346
lan 21
media_id 284
ntt 8
os 2
osv 154
package 1950
sid 500000
timestamp 500000
version 22
fea_hash 402980
location 332
fea1_hash 4959
cus_type 58


In [90]:
# fea_hash是否要做特征变换
#train['fea_hash'].value_counts()
#train['fea_hash'].describe()
train['fea_hash'].map(lambda x: len(str(x))).value_counts()
#train['fea1_hash'].value_counts()
train['fea1_hash'].map(lambda x: len(str(x))).value_counts()

10    391669
9      99347
8       8977
7          6
5          1
Name: fea1_hash, dtype: int64

In [91]:
# 对osv进行数据清洗
def osv_trans(x):
    x = str(x).replace('Android_', '').replace('Android ', '').replace('W', '')
    if str(x).find('.')>0:
        temp_index1 = x.find('.')
        if x.find(' ')>0:
            temp_index2 = x.find(' ')
        else:
            temp_index2 = len(x)

        if x.find('-')>0:
            temp_index2 = x.find('-')
            
        result = x[0:temp_index1] + '.' + x[temp_index1+1:temp_index2].replace('.', '')
        try:
            return float(result)
        except:
            print(x+'#########')
            return 0
    try:
        return float(x)
    except:
        print(x+'#########')
        return 0


In [92]:
# 采用众数，进行缺失值的填充
train['osv'].fillna('8.1.0', inplace=True)
# 数据清洗
train['osv'] = train['osv'].apply(osv_trans)
# 采用众数，进行缺失值的填充
test['osv'].fillna('8.1.0', inplace=True)
# 数据清洗
test['osv'] = test['osv'].apply(osv_trans)


f073b_changxiang_v01_b1b8_20180915#########
%E6%B1%9F%E7%81%B5OS+5.0#########
GIONEE_YNGA#########


In [93]:
#train['os'].value_counts()
train['lan'].value_counts()
#lan_map = {'zh-CN': 1, }
train['lan'].value_counts().index
lan_map = {'zh-CN': 1, 'zh_CN':2, 'Zh-CN': 3, 'zh-cn': 4, 'zh_CN_#Hans':5, 
           'zh': 6, 'ZH': 7, 'cn':8, 'CN':9, 'zh-HK': 10, 'tw': 11, 
           'TW': 12,'zh-TW': 13,'zh-MO':14, 'en':15, 'en-GB': 16, 
           'en-US': 17, 'ko': 18, 'ja': 19, 'it': 20, 'mi':21} 
train['lan'] = train['lan'].map(lan_map)
test['lan'] = test['lan'].map(lan_map)
test['lan'].value_counts()


1.0     73046
6.0      9094
8.0      6136
2.0      3391
3.0      2430
4.0       165
7.0        60
9.0        46
11.0       28
15.0       26
5.0        11
18.0        6
13.0        5
17.0        2
10.0        2
12.0        1
16.0        1
Name: lan, dtype: int64

In [94]:
# 对于有缺失的lan 设置为22
train['lan'].fillna(22, inplace=True)
test['lan'].fillna(22, inplace=True)


In [95]:
remove_list = ['Unnamed: 0','os',  'sid']
col = features
for i in remove_list:
    col.remove(i)
col

['android_id',
 'apptype',
 'carrier',
 'dev_height',
 'dev_ppi',
 'dev_width',
 'lan',
 'media_id',
 'ntt',
 'osv',
 'package',
 'timestamp',
 'version',
 'fea_hash',
 'location',
 'fea1_hash',
 'cus_type']

In [96]:
def version_trans(x):
    if x=='V3':
        return 3
    if x=='v1':
        return 1
    if x=='P_Final_6':
        return 6
    if x=='V6':
        return 6
    if x=='GA3':
        return 3
    if x=='GA2':
        return 2
    if x=='V2':
        return 2
    if x=='50':
        return 5
    return int(x)
train['version'] = train['version'].apply(version_trans)
test['version'] = test['version'].apply(version_trans)
train['version'] = train['version'].astype('int')
test['version'] = test['version'].astype('int')


In [97]:
#train['timestamp'].value_counts()
#train['timestamp'] = pd.to_datetime(train['timestamp'])
#train['timestamp']
from datetime import datetime

# lambda 是一句话函数，匿名函数
train['timestamp'] = train['timestamp'].apply(lambda x: datetime.fromtimestamp(x/1000))
#1559892728241.7212
#1559871800477.1477
#1625493942.538375
#import time
#time.time()
test['timestamp'] = test['timestamp'].apply(lambda x: datetime.fromtimestamp(x/1000))
test['timestamp']


0        2019-06-07 09:43:20.477148
1        2019-06-05 20:53:33.449314
2        2019-06-04 10:08:24.732394
3        2019-06-05 01:03:10.643892
4        2019-06-05 08:16:34.450110
                    ...            
149995   2019-06-08 09:21:33.629823
149996   2019-06-07 07:15:57.431662
149997   2019-06-09 08:48:59.661749
149998   2019-06-05 06:38:14.527250
149999   2019-06-08 07:23:16.149025
Name: timestamp, Length: 150000, dtype: datetime64[ns]

In [98]:
# 特征筛选
features = train[col]
# 构造fea_hash_len特征
features['fea_hash_len'] = features['fea_hash'].map(lambda x: len(str(x)))
features['fea1_hash_len'] = features['fea1_hash'].map(lambda x: len(str(x)))
# Thinking：为什么将很大的，很长的fea_hash化为0？
# 如果fea_hash很长，都归为0，否则为自己的本身
features['fea_hash'] = features['fea_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
features['fea1_hash'] = features['fea1_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
features


,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,lan,media_id,ntt,osv,package,timestamp,version,fea_hash,location,fea1_hash,cus_type,fea_hash_len,fea1_hash_len
0,316361,1199,46000.0,0.0,0.0,0.0,22.0,104,6.0,9.00,18,2019-06-07 15:32:08.241721,8,2135019403,0,2329670524,601,10,10
1,135939,893,0.0,0.0,0.0,0.0,22.0,19,6.0,8.10,0,2019-06-08 19:40:20.922229,4,2782306428,1,2864801071,1000,10,10
2,399254,821,0.0,760.0,0.0,360.0,22.0,559,0.0,8.10,0,2019-06-06 23:58:17.549109,0,1392806005,2,628911675,696,10,9
3,68983,1004,46000.0,2214.0,0.0,1080.0,22.0,129,2.0,8.10,0,2019-06-09 08:59:37.719685,0,3562553457,3,1283809327,753,10,10
4,288999,1076,46000.0,2280.0,0.0,1080.0,1.0,64,2.0,8.00,0,2019-06-07 08:28:21.478553,5,2364522023,4,1510695983,582,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,392477,1028,46000.0,1920.0,3.0,1080.0,1.0,144,6.0,7.12,25,2019-06-06 23:14:18.267242,7,861755946,79,140647032,373,9,9
499996,346134,1001,0.0,1424.0,0.0,720.0,22.0,29,2.0,8.10,0,2019-06-06 17:40:54.045280,3,1714444511,23,2745131047,525,10,10
499997,499635,761,46000.0,1280.0,0.0,720.0,22.0,54,6.0,6.01,9,2019-06-05 03:14:57.290064,0,3843262581,25,1326115882,810,10,10
499998,239786,917,46001.0,960.0,0.0,540.0,2.0,109,2.0,5.11,0,2019-06-07 00:59:27.651521,0,1984296118,225,1446741112,772,10,10


In [99]:
test_features = test[col]
# 构造fea_hash_len特征
test_features['fea_hash_len'] = test_features['fea_hash'].map(lambda x: len(str(x)))
test_features['fea1_hash_len'] = test_features['fea1_hash'].map(lambda x: len(str(x)))
# Thinking：为什么将很大的，很长的fea_hash化为0？
# 如果fea_hash很长，都归为0，否则为自己的本身
test_features['fea_hash'] = test_features['fea_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
test_features['fea1_hash'] = test_features['fea1_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
test_features


,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,lan,media_id,ntt,osv,package,timestamp,version,fea_hash,location,fea1_hash,cus_type,fea_hash_len,fea1_hash_len
0,317625,1181,46000.0,2196.0,2.0,1080.0,9.0,639,2.0,8.10,188,2019-06-07 09:43:20.477148,7,1672223856,57,3872258917,658,10,10
1,435108,944,46003.0,2280.0,3.0,1080.0,1.0,704,6.0,8.10,221,2019-06-05 20:53:33.449314,3,3767901757,23,129322164,943,10,9
2,0,1106,46000.0,0.0,0.0,0.0,22.0,39,2.0,5.10,1562,2019-06-04 10:08:24.732394,0,454638703,30,4226678391,411,9,10
3,451504,761,46000.0,1344.0,0.0,720.0,22.0,54,2.0,7.11,9,2019-06-05 01:03:10.643892,0,1507622951,65,3355419572,848,10,10
4,0,1001,46000.0,665.0,0.0,320.0,1.0,29,5.0,8.10,4,2019-06-05 08:16:34.450110,0,4116351093,148,2644467751,411,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,0,1001,46000.0,760.0,0.0,360.0,1.0,29,2.0,8.10,4,2019-06-08 09:21:33.629823,0,3162887451,126,2711576615,411,10,10
149996,0,1001,46000.0,780.0,0.0,360.0,1.0,29,2.0,9.00,4,2019-06-07 07:15:57.431662,0,97238959,322,2678022183,411,8,10
149997,0,1001,46000.0,780.0,0.0,360.0,1.0,29,5.0,8.10,4,2019-06-09 08:48:59.661749,0,1320118495,46,2610913319,411,10,10
149998,500925,1052,46000.0,854.0,240.0,480.0,1.0,249,6.0,4.42,0,2019-06-05 06:38:14.527250,2,1292986591,41,1898209327,430,10,10


In [100]:
#test['version'].value_counts()
#features['version'].value_counts()
features['dev_height'].value_counts()
features['dev_width'].value_counts()


1080.0    209914
0.0       107011
360.0      76852
720.0      63317
393.0       6259
           ...  
1368.0         1
1361.0         1
1357.0         1
1354.0         1
686.0          1
Name: dev_width, Length: 346, dtype: int64

In [101]:

# 构造面积特征
features['dev_area'] = features['dev_height'] * features['dev_width']
test_features['dev_area'] = test_features['dev_height'] * test_features['dev_width']
# 构造长宽比特征
features['height_width'] = features['dev_height'] / features['dev_width']
test_features['height_width'] = test_features['dev_height'] / test_features['dev_width']


In [102]:
#利用 dev_ppi 和 dev_area构造新特征，效果下降,不使用此特征
#features['dev_ppi'].value_counts()
'''
features['area_ppi'] = features['dev_area'].astype('float') / features['dev_ppi'].astype('float')
test_features['area_ppi'] = test_features['dev_area'].astype('float') / test_features['dev_ppi'].astype('float')
'''

"\nfeatures['area_ppi'] = features['dev_area'].astype('float') / features['dev_ppi'].astype('float')\ntest_features['area_ppi'] = test_features['dev_area'].astype('float') / test_features['dev_ppi'].astype('float')\n"

In [103]:
test_features.columns

Index(['android_id', 'apptype', 'carrier', 'dev_height', 'dev_ppi',
       'dev_width', 'lan', 'media_id', 'ntt', 'osv', 'package', 'timestamp',
       'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type',
       'fea_hash_len', 'fea1_hash_len', 'dev_area', 'height_width'],
      dtype='object')

In [104]:
features['version_osv'] = features['osv'] - features['version']
test_features['version_osv'] = test_features['osv'] - test_features['version']


In [105]:
# 对训练集的timestamp提取时间多尺度

# 使用to_datetime进行日期类型转换
# 创建时间戳索引
temp = pd.DatetimeIndex(features['timestamp'])
features['year'] = temp.year
features['month'] = temp.month
features['day'] = temp.day
features['week_day'] = temp.weekday #星期几
features['hour'] = temp.hour
features['minute'] = temp.minute


In [106]:
# 求时间的diff
start_time = features['timestamp'].min()
features['time_diff'] = features['timestamp'] - start_time
features['time_diff'] = features['time_diff'].dt.days + features['time_diff'].dt.seconds/3600/24
features[['timestamp', 'year', 'month', 'day', 'week_day', 'hour', 'minute', 'time_diff']]


,timestamp,year,month,day,week_day,hour,minute,time_diff
0,2019-06-07 15:32:08.241721,2019,6,7,4,15,32,4.647303
1,2019-06-08 19:40:20.922229,2019,6,8,5,19,40,5.819664
2,2019-06-06 23:58:17.549109,2019,6,6,3,23,58,3.998796
3,2019-06-09 08:59:37.719685,2019,6,9,6,8,59,6.374722
4,2019-06-07 08:28:21.478553,2019,6,7,4,8,28,4.353009
...,...,...,...,...,...,...,...,...
499995,2019-06-06 23:14:18.267242,2019,6,6,3,23,14,3.968252
499996,2019-06-06 17:40:54.045280,2019,6,6,3,17,40,3.736713
499997,2019-06-05 03:14:57.290064,2019,6,5,2,3,14,2.135370
499998,2019-06-07 00:59:27.651521,2019,6,7,4,0,59,4.041273


In [107]:
# 创建时间戳索引
temp = pd.DatetimeIndex(test_features['timestamp'])
test_features['year'] = temp.year
test_features['month'] = temp.month
test_features['day'] = temp.day
test_features['week_day'] = temp.weekday #星期几
test_features['hour'] = temp.hour
test_features['minute'] = temp.minute


In [108]:
# 求时间的diff
#start_time = features['timestamp'].min()
test_features['time_diff'] = test_features['timestamp'] - start_time
test_features['time_diff'] = test_features['time_diff'].dt.days + test_features['time_diff'].dt.seconds/3600/24
#test_features[['timestamp', 'year', 'month', 'day', 'week_day', 'hour', 'minute', 'time_diff']]
test_features['time_diff']


0         4.405081
1         2.870509
2         1.422488
3         2.043854
4         2.344826
            ...   
149995    5.389954
149996    4.302731
149997    6.367338
149998    2.276539
149999    5.307801
Name: time_diff, Length: 150000, dtype: float64

In [109]:
temp0 = train[train['label']==0]
temp = pd.DataFrame(columns=[0,1])
temp[0] = temp0['apptype'].value_counts() /len(temp0) *100


In [110]:
temp1 = train[train['label']==1]
temp[1] = temp1['apptype'].value_counts() /len(temp1) *100

In [111]:
#构造特征比
temp[2] = temp[1] /temp[0]
#筛选出来特征比大于10倍的特征
result = temp[temp[2] > 10 ].sort_values(2,ascending=False).index
result

Int64Index([1139, 716, 941, 851, 1034, 1067], dtype='int64')

In [112]:
#特征比 >10 效果好
def find_key_feature(train,selected):
    temp0 = train[train['label']==0]
    temp = pd.DataFrame(columns=[0,1])
    temp[0] = temp0[selected].value_counts() / len(temp0) * 100 #百分比

    temp1 =train[train['label']==1]
    temp[1] = temp1[selected].value_counts() / len(temp1) * 100 #百分比
    #构造特征比
    temp[2] = temp[1] /temp[0]
    #筛选出来特征比大于10倍的特征
    result = temp[temp[2] > 10].sort_values(2,ascending=False).index
    return result


In [113]:
key_feature = {}
key_feature['osv'] = find_key_feature(train,'osv')
key_feature


{'osv': Float64Index([   7.77,    7.21,    7.75,    7.85,    7.87,     3.8,    7.67,
                   3.9,     2.3,    8.01,     7.9,    3.84,    7.64,    7.89,
               21100.0,    7.92,     4.1,    7.72,    7.82,     7.8,    3.86,
                   7.7,    7.84,    7.68, 21000.0,    7.86,    7.73,     6.1,
                  3.83,    3.78,    7.74],
              dtype='float64')}

In [114]:
#将重要的特征值设置为1 否则为0
def f(x):
    if x in key_feature['osv']:
        return 1
    else:
        return 0


In [115]:
key_feature1 = {}
key_feature1['apptype'] = find_key_feature(train,'apptype')
key_feature1

{'apptype': Int64Index([1139, 716, 941, 851, 1034, 1067], dtype='int64')}

In [116]:
#将重要的特征值设置为1 否则为0
def f1(x):
    if x in key_feature1['apptype']:
        return 1
    else:
        return 0


In [117]:
features['osv1'] = features['osv'].apply(f)
test_features['osv1'] = test_features['osv'].apply(f)
features['osv1'].value_counts()


0    483294
1     16706
Name: osv1, dtype: int64

In [118]:
features['apptype1'] = features['apptype'].apply(f1)
test_features['apptype1'] = test_features['apptype'].apply(f1)
features['apptype1'].value_counts()


0    481614
1     18386
Name: apptype1, dtype: int64

In [119]:
'''
import re
features = features.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
test_features = test_features.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
'''

"\nimport re\nfeatures = features.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))\ntest_features = test_features.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))\n"

In [120]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
# 使用LGBM训练
import lightgbm as lgb
# 使用XGBM训练
import xgboost as xgb


In [121]:
#五折交叉验证
def lgb_model(clf,train_x,train_y,test):
    sk = StratifiedKFold(n_splits=5,random_state=2021,shuffle=True)
    mean_acc = 0
    prob = []
    #split时，传入feature和label
    for k,(train_index,val_index) in enumerate(sk.split(train_x,train_y)):
        #获取训练集特征和label
        train_x_real = train_x.iloc[train_index]
        train_y_real = train_y.iloc[train_index]
        #获取验证集特征和label
        val_x = train_x.iloc[val_index]
        val_y = train_y.iloc[val_index]
        #模型训练
        clf = clf.fit(train_x_real,train_y_real)
        #模型预测
        val_y_pred = clf.predict(val_x)
        acc_val = accuracy_score(val_y,val_y_pred)
        print('第{}个子模型 accuracy{}'.format(k+1,acc_val))
        mean_acc += acc_val / 5
        #对测试集进行预测
        test_y_pred =clf.predict_proba(test)
        prob.append(test_y_pred)
    print(mean_acc)
    mean_prob =sum(prob)/5
    return mean_prob


In [80]:
#使用五折子模型进行融合lgb
'''
model_lgb2 = lgb.LGBMClassifier(
            num_leaves=2**5-1, reg_alpha=0.25, reg_lambda=0.25, objective='binary',
            max_depth=-1, learning_rate=0.005, min_child_samples=3, random_state=2021,
            n_estimators=2000, subsample=1, colsample_bytree=1,
        )
'''
#使用五折子模型进行融合xgb
model_xgb = xgb.XGBClassifier(
            max_depth=15, learning_rate=0.005, n_estimators=2000, 
            objective='binary:logistic', tree_method='gpu_hist', 
            subsample=0.8, colsample_bytree=0.8, 
            min_child_samples=3, eval_metric='auc', reg_lambda=0.5
        )


In [81]:
test_features.columns

Index(['android_id', 'apptype', 'carrier', 'dev_height', 'dev_ppi',
       'dev_width', 'lan', 'media_id', 'ntt', 'osv', 'package', 'timestamp',
       'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type',
       'fea_hash_len', 'fea1_hash_len', 'dev_area', 'height_width',
       'version_osv', 'year', 'month', 'day', 'week_day', 'hour', 'minute',
       'time_diff', 'osv1', 'apptype1'],
      dtype='object')

In [82]:
%%time
#result_lgb2 = lgb_model(model_lgb2,features.drop(['timestamp','dev_height','dev_width'], axis=1), train['label'],test_features.drop(['timestamp','dev_height','dev_width'], axis=1))
result_lgb2 = lgb_model(model_xgb,features.drop(['timestamp','dev_height','dev_width'], axis=1), train['label'],test_features.drop(['timestamp','dev_height','dev_width'], axis=1))

result_lgb2


[23:51:28] WARNING: ../src/learner.cc:541: 
Parameters: { min_child_samples } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [83]:
def one_hot(x):
    if x < 0.5:
        x =0
    else:
        x =1
    return x


In [84]:
result_lgb_proba=[one_hot(x[1]) for x in result_lgb2]
result_lgb_proba
res = pd.DataFrame(test['sid'])
res['label'] = result_lgb_proba
res.to_csv('./add_dev_area_version_osv2_xgb_五折交叉验证2.csv',index=False)
res 
